# LFT: From Three Fundamental Laws to $A = L(I)$

**Goal.** Make explicit how the Three Fundamental Laws (Identity, Non-Contradiction, Excluded Middle) become a *generative operator* $L$ that filters the information space $I$ into actuality $A$.

We keep this notebook **dependency-free** and illustrative. Full-scale implementations (graphs, Cayley structures, permutohedra) appear in Notebooks 01–06.

## 1. Three Fundamental Laws (3FLL)
We model patterns as directed relations (ordered pairs) on a finite label set $E=\{0,\dots,N-1\}$. The laws act as constraints:

1. **Identity (ID)** — *Persistence / Reflexivity.*  
   Require $(i,i)$ for all $i\in E$. Intuition: self-identity across updates.

2. **Non-Contradiction (NC)** — *Acyclicity.*  
   Forbid directed cycles in the non-reflexive edges. Intuition: no contradictory preference loops.

3. **Excluded Middle (EM)** — *Decisiveness / Totalization when justified.*  
   When a relation is globally consistent (acyclic and definable), complete it to a **total order** (linear extension) if sufficient information exists.

These laws compose to a logical operator $L$.

## 2. Operator Composition
We define the **logical filter**:

$$ L \,=\, \mathrm{EM}\circ\mathrm{NC}\circ\mathrm{ID}, \qquad A = L(I). $$

- $\mathrm{ID}$: insert/preserve all reflexive edges.
- $\mathrm{NC}$: reject (map to $\bot$) any pattern with cycles.
- $\mathrm{EM}$: when feasible, complete to a total order; otherwise leave as a partial order.

Later notebooks implement $L$ *locally*, but this global composition captures the essence.

## 3. Minimal Algorithmic Schema (pure Python)
We provide a compact, dependency-free implementation that:

1. Adds reflexive edges (ID),
2. Checks for cycles (NC),
3. Attempts totalization (EM) by topological sorting when the edge set is complete for a total order.

> For clarity we separate **partial-order acceptance** from **totalization**; EM only totalizes when the number of non-reflexive edges equals $\binom{N}{2}$.

In [ ]:
from typing import List, Tuple, Optional, Set

Edge = Tuple[int,int]

def has_cycle(n:int, edges:Set[Edge]) -> bool:
    """DFS-based cycle detection on directed graph with nodes 0..n-1."""
    adj = {i:[] for i in range(n)}
    for a,b in edges:
        if a!=b:
            adj[a].append(b)
    state = [0]*n  # 0=unseen,1=visiting,2=done
    def dfs(u:int) -> bool:
        state[u]=1
        for v in adj[u]:
            if state[v]==1:
                return True
            if state[v]==0 and dfs(v):
                return True
        state[u]=2
        return False
    for i in range(n):
        if state[i]==0 and dfs(i):
            return True
    return False

def topo_sort_if_possible(n:int, edges:Set[Edge]) -> Optional[List[int]]:
    """Return one topological order if DAG, else None."""
    indeg = [0]*n
    adj = {i:[] for i in range(n)}
    for a,b in edges:
        if a!=b:
            adj[a].append(b); indeg[b]+=1
    order = []
    q = [i for i in range(n) if indeg[i]==0]
    while q:
        u = q.pop(0)
        order.append(u)
        for v in adj[u]:
            indeg[v]-=1
            if indeg[v]==0:
                q.append(v)
    return order if len(order)==n else None

def apply_L(n:int, nonref:List[Edge]):
    """Apply ID -> NC -> EM. Returns (status, pattern, order).
    status in { 'reject', 'partial', 'total' }.
    pattern is the reflexive-augmented edge set if accepted; order is a list if total.
    """
    # ID
    pattern = set(nonref) | {(i,i) for i in range(n)}
    # NC
    if has_cycle(n, pattern):
        return 'reject', None, None
    # EM: decide if we have a complete non-reflexive set for a total order
    expected = n*(n-1)//2
    nonref_only = {(a,b) for (a,b) in pattern if a!=b}
    if len(nonref_only)==expected:
        order = topo_sort_if_possible(n, nonref_only)
        if order is not None:
            return 'total', pattern, order
        else:
            return 'reject', None, None
    return 'partial', pattern, None

# Quick sanity examples (N=4)
n=4
print(apply_L(n, [(0,1),(1,2)]))              # partial
print(apply_L(n, [(0,1),(1,2),(2,3),(0,2),(1,3),(0,3)]))  # total
print(apply_L(n, [(0,1),(1,0)]))              # reject (cycle)


## 4. Correctness Intuitions
- **ID → NC:** Reflexivity ensures stable identity; NC removes contradictions (cycles).  
- **NC → EM:** In a DAG on $N$ nodes with $\binom{N}{2}$ non-reflexive edges, EM checks if the DAG is a **linear order** (one topological order).  
- **Outcomes:**
  - **reject:** violates NC, or EM fails due to inconsistency.
  - **partial:** consistent but not fully decisive.
  - **total:** consistent and decisive → corresponds to a **permutation**.


## 5. Why this yields permutation geometry (preview)
Applying $L$ to all patterns picks out **acyclic totals** which correspond one-to-one with permutations (linear extensions).  
Thus $S_N$ and its **$N-1$ adjacent generators** (Coxeter type $A_{N-1}$) appear *inevitably* from the logical program—not as an external ansatz.  

Notebook **02_Geometry** formalizes the sum-zero space $V\cong\mathbb{R}^{N-1}$ and the **permutohedron** $\Pi_{N-1}$, showing **rank = $N-1$** as intrinsic dimension.

## 6. Hand-off to later notebooks
- **01_Introduction**: enumeration at small $N$ and filtering statistics (cycles/partials/totals).
- **02_Geometry**: $A_{N-1}$ geometry; permutohedra and adjacent-edge Cayley graphs.
- **03_Dynamics\_Time**: inversion count $h$ as Lyapunov; time as $L$-flow.
- **04_Quantum\_Link**: simplex orbits and the qudit bridge.
- **05–06**: scaling to $N=6$ and spacetime factorization.